In [ ]:
# Import the dependencies
import kagglehub
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import random
import matplotlib.image as mpimg
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
# Download latest version Dataset from kaggle
path = kagglehub.dataset_download("gpiosenka/cards-image-datasetclassification")

print("Path to dataset files:", path)

In [ ]:
# Display train , test and validation set with its data
def simple_dataset_info(path):
    for split in ['train', 'valid', 'test']:
        split_path = os.path.join(path, split)
        if not os.path.exists(split_path):
            print(f"{split} folder not found.")
            continue

        classes = [d for d in os.listdir(split_path) if os.path.isdir(os.path.join(split_path, d))]
        total_images = 0

        for cls in classes:
            class_path = os.path.join(split_path, cls)
            num_images = len(os.listdir(class_path))
            total_images += num_images

        print(f"{split.upper()}:")
        print(f"  Classes: {len(classes)}")
        print(f"  Total Images: {total_images}\n")

# Run it
simple_dataset_info(path)


In [ ]:
# Display training data images and labels
def show_random_images(split='train', num_images=6):
    split_path = os.path.join(path, split)
    classes = [d for d in os.listdir(split_path) if os.path.isdir(os.path.join(split_path, d))]

    plt.figure(figsize=(7,7))
    for i in range(num_images):
        random_class = random.choice(classes)
        class_path = os.path.join(split_path, random_class)
        image_file = random.choice(os.listdir(class_path))
        img_path = os.path.join(class_path, image_file)

        img = mpimg.imread(img_path)
        plt.subplot(3, 3, i+1)
        plt.imshow(img)
        plt.title(random_class)
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Example
show_random_images('train')


In [ ]:
img_size = (128, 128)

In [ ]:

def load_data(split):
    X = []
    y = []
    split_path = os.path.join(path, split)
    class_names = sorted(os.listdir(split_path))  # Sorted to have consistent label mapping
    class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}

    for c in class_names:
        c_path = os.path.join(split_path, c)
        if not os.path.isdir(c_path):
            continue
        for img_name in os.listdir(c_path):
            img_path = os.path.join(c_path, img_name)
            try:
                img = Image.open(img_path).convert('RGB')
                img = img.resize(img_size)
                img = np.array(img) / 255.0  # normalize to [0, 1]
                X.append(img)
                y.append(class_to_idx[c])
            except:
                print(f"Error loading {img_path}")

    return np.array(X), np.array(y), class_to_idx


In [ ]:
X_train, y_train, class_map = load_data('train')
X_valid, y_valid, _ = load_data('valid')
X_test, y_test, _ = load_data('test')

print(f"Train: {X_train.shape}, Labels: {y_train.shape}")
print(f"Valid: {X_valid.shape}, Labels: {y_valid.shape}")
print(f"Test: {X_test.shape}, Labels: {y_test.shape}")

In [ ]:
X_train[10]

In [ ]:
input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))

In [ ]:
from tensorflow.keras.utils import to_categorical

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes)
y_valid = to_categorical(y_valid, num_classes)
y_test = to_categorical(y_test, num_classes)


In [ ]:
# Create The Cnn model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu',),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu',),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])


In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=10,
    batch_size=16
)

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


In [ ]:
#Activate to create folders to your google colab

"""
from google.colab import drive
drive.mount('/content/drive')

import os
result_dir = '/content/drive/MyDrive/re'
model_dir = '/content/drive/MyDrive/ImageClassification'

# Create dir if not exist
os.makedirs(result_dir , exist_ok=True)
os.makedirs(model_dir , exist_ok=True)

"""

In [ ]:
# Save The model
# ( Activate if you want to save the model )
"""
model.save(f"{model_dir}/card.h5")
from tensorflow.keras.models import load_model
# Load the saved model
Loaded_model = load_model(f"{model_dir}/card.h5")

"""

In [ ]:
# Predict on the test set
predictions = model.predict(X_test)

# Get the predicted class index label
predicted_classes = np.argmax(predictions, axis=1)

# Get the true class labels
true_classes = np.argmax(y_test, axis=1)


In [ ]:
train_path = os.path.join(path, 'train')
class_names = sorted(os.listdir(train_path))

fig, axes = plt.subplots(2, 3, figsize=(7,7))

# Choose 6 random indices for the test set
random_indices = random.sample(range(len(X_test)), 6)

for idx, ax in zip(random_indices, axes.flat):
    ax.imshow(X_test[idx])
    ax.set_title(f"Pred: {class_names[predicted_classes[idx]]}\nTrue: {class_names[true_classes[idx]]}")
    ax.axis('off')

plt.tight_layout()
plt.show()